In [1]:
# Enable outputting results interactively. 
# All the results from code in a given cell will be displayed instead of only the last one.

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import re
from datetime import datetime

from bs4 import BeautifulSoup, SoupStrainer
import requests
import pickle

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
url = "http://insideairbnb.com/get-the-data.html"

page = requests.get(url)    
data = page.text
soup = BeautifulSoup(data)

links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))

In [4]:
def gather_links(city,cat):

    len(links)
    links_df = pd.DataFrame(links, columns = ['link'])
    links_df2 = links_df[links_df['link'].str.contains(city)==True]
    links_df3 = links_df2[links_df2['link'].str.contains(cat)==True]
    return links_df3

In [5]:
def readin_gz(dt,cat):
    url = 'http://data.insideairbnb.com/united-states/ca/san-francisco/{}/data/{}.csv.gz'.format(str(dt),str(cat))

    listings = pd.read_csv(url,compression='gzip',header=0, sep=',', quotechar='"',dtype= {'zipcode':object})
    return listings
    

In [6]:
links_sf_listings = gather_links('san-francisco','listings.csv.gz')
links_sf_listings['dt'] = [x.split('/')[6] for x in links_sf_listings['link']]
links_sf_listings.head()

idx = links_sf_listings['dt'].unique()
len(idx),idx

,link,dt
7529,http://data.insideairbnb.com/united-states/ca/...,2019-05-03
7536,http://data.insideairbnb.com/united-states/ca/...,2019-04-03
7543,http://data.insideairbnb.com/united-states/ca/...,2019-03-06
7550,http://data.insideairbnb.com/united-states/ca/...,2019-02-01
7557,http://data.insideairbnb.com/united-states/ca/...,2019-01-09


(45, array(['2019-05-03', '2019-04-03', '2019-03-06', '2019-02-01',
        '2019-01-09', '2018-12-06', '2018-11-03', '2018-10-03',
        '2018-09-08', '2018-08-06', '2018-07-05', '2018-05-09',
        '2018-04-06', '2018-03-04', '2018-02-02', '2018-01-17',
        '2018-01-10', '2017-12-07', '2017-12-02', '2017-11-08',
        '2017-11-01', '2017-10-02', '2017-09-02', '2017-08-02',
        '2017-07-02', '2017-06-02', '2017-05-02', '2017-04-02',
        '2017-03-02', '2017-02-02', '2017-01-01', '2016-12-03',
        '2016-11-02', '2016-10-01', '2016-09-02', '2016-08-02',
        '2016-07-02', '2016-06-02', '2016-05-02', '2016-04-03',
        '2016-02-02', '2015-12-02', '2015-11-01', '2015-09-02',
        '2015-05-04'], dtype=object))

In [7]:
listings = readin_gz(idx[0],'listings')    
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7353 entries, 0 to 7352
Columns: 106 entries, id to reviews_per_month
dtypes: float64(23), int64(21), object(62)
memory usage: 5.9+ MB


In [8]:


for i, dt in enumerate(idx[1:]):
    df = readin_gz(idx[i+1],'listings')
    print(dt,df['last_scraped'].value_counts())
    listings = pd.concat([listings, df], ignore_index = True,axis = 0,sort=False)
        

2019-04-03 2019-04-03    7276
2019-04-05       1
Name: last_scraped, dtype: int64
2019-03-06 2019-03-06    7151
Name: last_scraped, dtype: int64
2019-02-01 2019-02-01    7198
Name: last_scraped, dtype: int64
2019-01-09 2019-01-09    7251
Name: last_scraped, dtype: int64
2018-12-06 2018-12-06    7072
Name: last_scraped, dtype: int64
2018-11-03 2018-11-03    6981
Name: last_scraped, dtype: int64
2018-10-03 2018-10-03    6807
Name: last_scraped, dtype: int64
2018-09-08 2018-09-08    6529
Name: last_scraped, dtype: int64
2018-08-06 2018-08-06    6633
Name: last_scraped, dtype: int64
2018-07-05 2018-07-05    4693
Name: last_scraped, dtype: int64
2018-05-09 2018-05-09    4774
Name: last_scraped, dtype: int64
2018-04-06 2018-04-07    3593
2018-04-06    1251
Name: last_scraped, dtype: int64
2018-03-04 2018-03-04    4802
2018-03-06       2
Name: last_scraped, dtype: int64
2018-02-02 2018-02-02    4727
Name: last_scraped, dtype: int64
2018-01-17 2018-01-17    5224
Name: last_scraped, dtype: int6

In [9]:
listings['last_scraped'].value_counts()

2016-12-03    9063
2017-08-02    9057
2017-01-01    9035
2017-10-02    8933
2017-11-01    8929
2016-11-02    8864
2017-07-02    8854
2017-06-02    8799
2017-02-02    8777
2016-10-01    8759
2017-03-02    8720
2017-04-02    8706
2016-09-02    8665
2016-07-02    8619
2017-05-02    8608
2017-09-02    8529
2016-08-02    8501
2016-06-02    8440
2016-02-02    8299
2016-05-02    8086
2016-04-03    8012
2019-05-03    7353
2019-04-03    7276
2019-01-09    7251
2019-02-01    7198
2015-12-02    7165
2019-03-06    7151
2018-12-06    7072
2018-11-03    6981
2017-11-08    6866
2018-10-03    6807
2017-12-02    6788
2017-12-07    6650
2018-08-06    6633
2018-09-08    6529
2018-01-10    6506
2015-09-02    6361
2015-11-02    6089
2018-01-17    5224
2018-03-04    4802
2018-05-09    4774
2018-02-02    4727
2018-07-05    4693
2015-05-04    3750
2018-04-07    3593
2015-05-05    1675
2018-04-06    1251
2015-11-01     940
2017-05-03     162
2017-12-03     152
2016-08-03     109
2016-12-04       3
2018-03-06  

In [10]:
ls_cols = listings.columns

listings['is_duplicated'] = listings.duplicated(ls_cols)
listings['is_duplicated'].sum()

listings_u = listings[listings['is_duplicated'] == False].drop(['is_duplicated'],axis=1)
listings_u.shape

0

(334790, 106)

In [11]:
ls_cols[43:44]

Index(['zipcode'], dtype='object')

In [12]:
listings_u = listings_u.set_index(pd.DatetimeIndex(listings_u['last_scraped']))

d_ls = listings_u.resample('D').size().reset_index(name='Counts') 
d_ls[d_ls['Counts']>0]

,last_scraped,Counts
0,2015-05-04,3750
1,2015-05-05,1675
121,2015-09-02,6361
181,2015-11-01,940
182,2015-11-02,6089
212,2015-12-02,7165
274,2016-02-02,8299
335,2016-04-03,8012
364,2016-05-02,8086
395,2016-06-02,8440


In [13]:
pickle.dump(listings_u, open('all_SF_listings.p','wb'))

In [14]:
listings.shape

(334790, 107)

In [15]:
links_sf_reviews = gather_links('san-francisco','reviews.csv.gz')
links_sf_reviews['dt'] = [x.split('/')[6] for x in links_sf_reviews['link']]
links_sf_reviews.head()

idx = links_sf_reviews['dt'].unique()
len(idx),idx

,link,dt
7531,http://data.insideairbnb.com/united-states/ca/...,2019-05-03
7538,http://data.insideairbnb.com/united-states/ca/...,2019-04-03
7545,http://data.insideairbnb.com/united-states/ca/...,2019-03-06
7552,http://data.insideairbnb.com/united-states/ca/...,2019-02-01
7559,http://data.insideairbnb.com/united-states/ca/...,2019-01-09


(45, array(['2019-05-03', '2019-04-03', '2019-03-06', '2019-02-01',
        '2019-01-09', '2018-12-06', '2018-11-03', '2018-10-03',
        '2018-09-08', '2018-08-06', '2018-07-05', '2018-05-09',
        '2018-04-06', '2018-03-04', '2018-02-02', '2018-01-17',
        '2018-01-10', '2017-12-07', '2017-12-02', '2017-11-08',
        '2017-11-01', '2017-10-02', '2017-09-02', '2017-08-02',
        '2017-07-02', '2017-06-02', '2017-05-02', '2017-04-02',
        '2017-03-02', '2017-02-02', '2017-01-01', '2016-12-03',
        '2016-11-02', '2016-10-01', '2016-09-02', '2016-08-02',
        '2016-07-02', '2016-06-02', '2016-05-02', '2016-04-03',
        '2016-02-02', '2015-12-02', '2015-11-01', '2015-09-02',
        '2015-05-04'], dtype=object))

In [16]:
reviews = readin_gz(idx[0],'reviews')       

for i, dt in enumerate(idx[1:]):
    df = readin_gz(idx[i+1],'reviews')
    print(dt)
    reviews = pd.concat([reviews, df], ignore_index = True,axis = 0,sort=False)
        

2019-04-03
2019-03-06
2019-02-01
2019-01-09
2018-12-06
2018-11-03
2018-10-03
2018-09-08
2018-08-06
2018-07-05
2018-05-09
2018-04-06
2018-03-04
2018-02-02
2018-01-17
2018-01-10
2017-12-07
2017-12-02
2017-11-08
2017-11-01
2017-10-02
2017-09-02
2017-08-02
2017-07-02
2017-06-02
2017-05-02
2017-04-02
2017-03-02
2017-02-02
2017-01-01
2016-12-03
2016-11-02
2016-10-01
2016-09-02
2016-08-02
2016-07-02
2016-06-02
2016-05-02
2016-04-03
2016-02-02
2015-12-02
2015-11-01
2015-09-02
2015-05-04


In [17]:
reviews.shape

(10302048, 6)

In [18]:
rv_cols = reviews.columns

In [19]:
reviews['is_duplicated'] = reviews.duplicated(rv_cols)
reviews['is_duplicated'].sum()

reviews_u = reviews[reviews['is_duplicated'] == False].drop(['is_duplicated'],axis=1)
reviews_u.shape

9651267

(650781, 6)

In [20]:
reviews_u.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...


In [21]:
pickle.dump(reviews_u, open('all_SF_reviews.p','wb'))

In [22]:
links_sf_calendar = gather_links('san-francisco','calendar.csv.gz')
links_sf_calendar['dt'] = [x.split('/')[6] for x in links_sf_calendar['link']]
links_sf_calendar.head()

idx = links_sf_calendar['dt'].unique()
len(idx),idx

,link,dt
7530,http://data.insideairbnb.com/united-states/ca/...,2019-05-03
7537,http://data.insideairbnb.com/united-states/ca/...,2019-04-03
7544,http://data.insideairbnb.com/united-states/ca/...,2019-03-06
7551,http://data.insideairbnb.com/united-states/ca/...,2019-02-01
7558,http://data.insideairbnb.com/united-states/ca/...,2019-01-09


(45, array(['2019-05-03', '2019-04-03', '2019-03-06', '2019-02-01',
        '2019-01-09', '2018-12-06', '2018-11-03', '2018-10-03',
        '2018-09-08', '2018-08-06', '2018-07-05', '2018-05-09',
        '2018-04-06', '2018-03-04', '2018-02-02', '2018-01-17',
        '2018-01-10', '2017-12-07', '2017-12-02', '2017-11-08',
        '2017-11-01', '2017-10-02', '2017-09-02', '2017-08-02',
        '2017-07-02', '2017-06-02', '2017-05-02', '2017-04-02',
        '2017-03-02', '2017-02-02', '2017-01-01', '2016-12-03',
        '2016-11-02', '2016-10-01', '2016-09-02', '2016-08-02',
        '2016-07-02', '2016-06-02', '2016-05-02', '2016-04-03',
        '2016-02-02', '2015-12-02', '2015-11-01', '2015-09-02',
        '2015-05-04'], dtype=object))

In [23]:
calendar = readin_gz(idx[0],'calendar')       

for i, dt in enumerate(idx[1:]):
    df = readin_gz(idx[i],'calendar')
    print(dt)
    calendar = pd.concat([calendar, df], ignore_index = True,axis = 0,sort=False)

2019-04-03
2019-03-06
2019-02-01
2019-01-09
2018-12-06
2018-11-03
2018-10-03
2018-09-08
2018-08-06
2018-07-05
2018-05-09
2018-04-06
2018-03-04
2018-02-02
2018-01-17
2018-01-10
2017-12-07
2017-12-02
2017-11-08
2017-11-01
2017-10-02
2017-09-02
2017-08-02
2017-07-02
2017-06-02
2017-05-02
2017-04-02
2017-03-02
2017-02-02
2017-01-01
2016-12-03
2016-11-02
2016-10-01
2016-09-02
2016-08-02
2016-07-02
2016-06-02
2016-05-02
2016-04-03
2016-02-02
2015-12-02
2015-11-01
2015-09-02
2015-05-04


In [24]:
calendar.shape

(122904625, 7)

In [25]:
cal_cols = calendar.columns

In [26]:
calendar['is_duplicated'] = calendar.duplicated(cal_cols)
calendar['is_duplicated'].sum()

calendar_u = calendar[calendar['is_duplicated'] == False].drop(['is_duplicated'],axis=1)
calendar_u.shape

83621290

(39283335, 7)

In [27]:
calendar_u.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,34158472,2019-05-03,f,$399.00,$399.00,2.0,1125.0
1,34158472,2019-05-04,f,$399.00,$399.00,2.0,1125.0
2,34158472,2019-05-05,t,$399.00,$399.00,2.0,1125.0
3,34158472,2019-05-06,t,$399.00,$399.00,2.0,1125.0
4,34158472,2019-05-07,t,$399.00,$399.00,2.0,1125.0


In [28]:
pickle.dump(calendar_u, open('all_SF_calendar.p','wb'))

In [29]:
# listings['has_availability'].value_counts()

In [30]:
#  pd.crosstab(listings_u['property_type'].str.upper(), listings_u['room_type'].str.upper(), margins=True)

In [31]:
# len(listings_u['host_id'].unique())

In [32]:
# len(listings_u['id'].unique())

In [33]:
# listings_u['neighbourhood'].str.upper().value_counts().reset_index()

In [34]:
# listings_u['zipcode'] = listings_u['zipcode'].astype(str).str.zfill(5)[:-2]

In [35]:
# listings_u = listings_u.set_index(pd.DatetimeIndex(listings_u['last_scraped']))

In [36]:
# m_ls = listings_u.resample('M').size().reset_index(name='Counts') 
# m_ls = m_ls[m_ls['Counts']>0]
# m_ls

In [37]:
# pd.crosstab(listings_u['last_scraped'], listings_u['has_availability'],margins=True)